In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from tqdm import tqdm

### Generating data

In [45]:
RANDOM_STATE = 42

In [ ]:
def my_make_classification(
    n_samples=1000, n_features=10, n_informative=4, random_state=42
):

    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative,
        random_state=random_state,
    )
    X = pd.DataFrame(X)
    y = pd.Series(y)
    X.columns = [f"col_{col}" for col in X.columns]
    return X, y

In [54]:
X, target = my_make_classification()
print(X.shape, target.shape)

numerical = X.columns
print(numerical)

(1000, 10) (1000,)
Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9'],
      dtype='object')


In [ ]:
df = X
df['y'] = target
df.sample(5)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,y
352,1.013258,-1.258270,1.472812,0.488919,-2.162642,-0.179325,0.311137,0.438655,-0.808902,0.792976,0
714,0.527524,-0.049019,3.150097,-0.505074,-2.525122,-2.644007,0.976627,1.528321,0.551160,0.801472,1
492,1.620306,-0.692802,1.374491,0.091258,-2.199630,0.020042,0.477815,1.077813,0.334211,1.288174,0
186,-0.643272,0.111040,2.714094,2.114175,-1.274332,-1.221178,0.010230,-1.085825,0.138405,1.296911,1
696,-2.849537,-0.538835,-1.320915,0.243763,0.478186,-0.115585,-0.978731,-0.057837,-1.597796,-4.065120,0


### Individual gini

In [66]:
train_gini, valid_gini = train_test_split(df[numerical], test_size=0.2, stratify=df['y'], random_state=RANDOM_STATE)

In [ ]:
# numeral = list of numerical features
gini_fiatures = pd.DataFrame(index=numerical, columns=["gini_train", "gini_valid"])

train_gini = train_gini.join(df['y'], how='left')
valid_gini = valid_gini.join(df['y'], how='left')

for feat in tqdm(numerical):
    logreg = LogisticRegression(random_state=RANDOM_STATE, penalty="l2", C=1)
    logreg.fit(train_gini.loc[:, feat].values.reshape(-1, 1), train_gini["y"])
    preds_train = logreg.predict_proba(train_gini.loc[:, feat].values.reshape(-1, 1)[:, 1])
    preds_valid = logreg.predict_proba(valid_gini.loc[:, feat].values.reshape(-1, 1)[:, 1])
    gini_features.loc[feat, 'gini_train'] = 2 * roc_auc_score(train_gini["y"], preds_train) - 1
    gini_features.loc[feat, 'gini_valid'] = 2 * roc_auc_score(valid_gini["y"], preds_train) - 1


ValueError: columns overlap but no suffix specified: Index(['y'], dtype='object')